In [8]:
import vtk
import numpy as np
from vtk.util import numpy_support
import matplotlib.pyplot as plt

In [2]:
object_path = "./assets/cat1_un4.obj"

reader = vtk.vtkOBJReader()
reader.SetFileName(object_path)
reader.Update()

# --- Step 2: Create a mapper ---
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(reader.GetOutputPort())

# --- Step 3: Create an actor ---
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Optional: Apply some color and lighting properties
actor.GetProperty().SetColor(1.0, 1.0, 1.0)   # White
actor.GetProperty().SetSpecular(0.5)
actor.GetProperty().SetSpecularPower(20)



In [3]:
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# --- Step 5: Add actor and background ---
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)  # Dark gray background

# --- Step 6: Start rendering ---
render_window.Render()
interactor.Start()

In [10]:
# --- Step 1: Load vertices from file ---
coordinates_file_path = "./assets/Objet3D.txt"

coordinates = []
with open(coordinates_file_path, "r") as file:
    for line in file:
        if line.startswith("v "):  # Only process vertex lines
            _, x, y, z = line.strip().split()
            coordinates.append([float(x), float(y), float(z)])

points = np.array(coordinates)
print(f"Loaded {len(points)} points.")

# --- Step 2: Convert numpy array to VTK points ---
vtk_points = vtk.vtkPoints()
for p in points:
    vtk_points.InsertNextPoint(p.tolist())

Loaded 4008 points.


In [ ]:
# --- Step 3: Perform 3D Delaunay triangulation ---
delaunay = vtk.vtkDelaunay3D()
polydata = vtk.vtkPolyData()
polydata.SetPoints(vtk_points)
delaunay.SetInputData(polydata)
delaunay.Update()

# --- Step 4: Extract the surface of the triangulated volume ---
surface_filter = vtk.vtkDataSetSurfaceFilter()
surface_filter.SetInputConnection(delaunay.GetOutputPort())
surface_filter.Update()

# --- Step 5: Create mapper and actor ---
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(surface_filter.GetOutputPort())

actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(0.8, 0.8, 0.9)
actor.GetProperty().SetOpacity(1.0)
actor.GetProperty().EdgeVisibilityOn()

# --- Step 6: Create renderer, window, and interactor ---
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)

render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.SetSize(800, 600)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# --- Step 7: Start interaction ---
render_window.Render()
interactor.Start()

: 